In [2]:
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.circuit.library import HartreeFock
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 
from qiskit.algorithms import VQE
from IPython.display import display, clear_output
# Store results in a dictionary
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller
from qiskit_nature.transformers import FreezeCoreTransformer


# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

ibmqfactory.load_account:WARNING:2021-06-08 12:38:10,005: Credentials are already in use. The existing account in the session will be replaced.


In [3]:
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()
freezeCoreTransfomer = FreezeCoreTransformer(True, [3, 4])
# [3, 4] are removed by checking Z2 symmetries as in next cell
# By using different combinations, we checked which combination
# resulted in 0 Z2 symmetries remaining
# In this specific case, no Z2 symmetries were left as will be clear in next cell

qmolecule = freezeCoreTransfomer.transform(qmolecule)

In [4]:
problem = ElectronicStructureProblem(driver, q_molecule_transformers=[freezeCoreTransfomer])
# Generate the second-quantized operators
second_q_ops = problem.second_q_ops()
# Hamiltonian
main_op = second_q_ops[0]
#print(second_q_ops)

# Setup the mapper and qubit converter
mapper_type = 'ParityMapper'

if mapper_type == 'ParityMapper':
    mapper = ParityMapper()
elif mapper_type == 'JordanWignerMapper':
    mapper = JordanWignerMapper()
elif mapper_type == 'BravyiKitaevMapper':
    mapper = BravyiKitaevMapper()

converter = QubitConverter(mapper=mapper, two_qubit_reduction=True, z2symmetry_reduction=None)

# The fermionic operators are mapped to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
qubit_op = converter.convert(main_op, num_particles=num_particles)

# We change z2symmetry_reduction parameter to 'auto' in converter
# And then we run the next line
print(converter.z2symmetries)
# The above statement prints all the Z2 symmetries
# And gives us ideas for combinations in previous cell

print(qubit_op) # Shows us that our problem now has only 4 orbitals


num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals

init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(init_state)

Z2 symmetries:
Symmetries:
Single-Qubit Pauli X:
Cliffords:
Qubit index:
[]
Tapering values:
  - Possible values: []
-0.20316606150558764 * IIII
+ (-0.36525869021603974-1.3877787807814457e-17j) * ZIII
+ 0.09275994933497558 * IZII
- 0.21188984297008465 * ZZII
+ (0.36525869021603957+2.7755575615628914e-17j) * IIZI
- 0.11384335176463774 * ZIZI
+ 0.11395251883045748 * IZZI
+ (-0.0604401285731684-3.469446951953614e-18j) * ZZZI
- 0.09275994933497578 * IIIZ
+ 0.1139525188304575 * ZIIZ
+ (-0.1227424405254314-6.938893903907228e-18j) * IZIZ
+ 0.05628878167218642 * ZZIZ
+ (-0.21188984297008465+2.7755575615628914e-17j) * IIZZ
+ 0.0604401285731684 * ZIZZ
+ (-0.056288781672186414+3.469446951953614e-18j) * IZZZ
+ 0.08460131391823414 * ZZZZ
+ 0.01938940858368941 * XIII
- 0.01938940858368941 * XZII
+ (-0.010952773573819184+8.673617379884035e-19j) * XIZI
+ 0.010952773573819182 * XZZI
+ 0.012779333033036876 * XIIZ
- 0.012779333033036877 * XZIZ
- 0.009002501243838454 * XIZZ
+ 0.009002501243838454 * XZZZ
+

In [5]:
from qiskit.circuit.library import TwoLocal, EfficientSU2, RealAmplitudes
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

# Although other ansatz options are given, our final results were given by TwoLocal

# Explanation of Method used:
# Basically, I started to use normal TwoLocal first and tried varying
# Type of entanglement and Number of repitions
# Reducing the number of repitions, I got good result even for repitions=2
# And even linear entanglement worked
# This resulted in a cost of 6, now next goal was to reduce it even further
# From here, I started using custom entanglement in TwoLocal itself
# Starting from linear entanglement, I kept on reducing the CNOT gates
# That were not required to give us good results
# Also, since linear entanglemnt didn't covered all CNOT combinations
# Like [0, 3] CNOT gate is not part of linear entanglement
# I tried adding such combinations as well which lead me to
# The final cost of 3 with custom entanglement as shown below


# Choose the ansatz
ansatz_type = "TwoLocal"

# Parameters for q-UCC antatze
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals

# Put arguments for twolocal
if ansatz_type == "TwoLocal":
    # Single qubit rotations that are placed on all qubits with independent parameters
    rotation_blocks = ['rz', 'ry']
    # Entangling gates
    entanglement_blocks = 'cx'
    # How the qubits are entangled 
    entanglement = 'linear'
    entanglement = [[(0, 1), (0, 3)], [(1, 2)]]
    # Repetitions of rotation_blocks + entanglement_blocks with independent parameters
    repetitions = 2
    # Skip the final rotation_blocks layer
    skip_final_rotation_layer = True
    ansatz = TwoLocal(qubit_op.num_qubits, rotation_blocks, entanglement_blocks, reps=repetitions, 
                      entanglement=entanglement, skip_final_rotation_layer=skip_final_rotation_layer)
    #ansatz = EfficientSU2(qubit_op.num_qubits, reps=repetitions, 
                      #entanglement=entanglement, skip_final_rotation_layer=skip_final_rotation_layer)
    # Add the initial state
    ansatz.compose(init_state, front=True, inplace=True)
elif ansatz_type == "UCCSD":
    ansatz = UCCSD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "PUCCD":
    ansatz = PUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "SUCCD":
    ansatz = SUCCD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
elif ansatz_type == "Custom":
    # Example of how to write your own circuit
    from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister
    # Define the variational parameter
    theta = Parameter('a')
    n = qubit_op.num_qubits
    # Make an empty quantum circuit
    qc = QuantumCircuit(qubit_op.num_qubits)
    qubit_label = 0
    # Place a Hadamard gate
    qc.h(qubit_label)
    # Place a CNOT ladder
    for i in range(n-1):
        qc.cx(i, i+1)
    # Visual separator
    qc.barrier()
    # rz rotations on all qubits
    qc.rz(theta, range(n))
    ansatz = qc
    ansatz.compose(init_state, front=True, inplace=True)

print(ansatz)


        ┌───┐    ┌──────────┐ ┌──────────┐                              »
q_0: ───┤ X ├────┤ RZ(θ[0]) ├─┤ RY(θ[4]) ├──────■────────────────────■──»
        ├───┤    ├──────────┤ ├──────────┤    ┌─┴─┐    ┌──────────┐  │  »
q_1: ───┤ X ├────┤ RZ(θ[1]) ├─┤ RY(θ[5]) ├────┤ X ├────┤ RZ(θ[9]) ├──┼──»
     ┌──┴───┴───┐├──────────┤┌┴──────────┤┌───┴───┴───┐└──────────┘  │  »
q_2: ┤ RZ(θ[2]) ├┤ RY(θ[6]) ├┤ RZ(θ[10]) ├┤ RY(θ[14]) ├──────────────┼──»
     ├──────────┤├──────────┤└───────────┘└───────────┘            ┌─┴─┐»
q_3: ┤ RZ(θ[3]) ├┤ RY(θ[7]) ├──────────────────────────────────────┤ X ├»
     └──────────┘└──────────┘                                      └───┘»
«      ┌──────────┐┌───────────┐
«q_0: ─┤ RZ(θ[8]) ├┤ RY(θ[12]) ├
«     ┌┴──────────┤└───────────┘
«q_1: ┤ RY(θ[13]) ├──────■──────
«     └───────────┘    ┌─┴─┐    
«q_2: ─────────────────┤ X ├────
«     ┌───────────┐┌───┴───┴───┐
«q_3: ┤ RZ(θ[11]) ├┤ RY(θ[15]) ├
«     └───────────┘└───────────┘


In [6]:
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

optimizer_type = 'SLSQP'

# You may want to tune the parameters 
# of each optimizer, here the defaults are used
if optimizer_type == 'COBYLA':
    optimizer = COBYLA(maxiter=5000)
elif optimizer_type == 'L_BFGS_B':
    optimizer = L_BFGS_B(maxfun=5000)
elif optimizer_type == 'SPSA':
    optimizer = SPSA(maxiter=5000)
elif optimizer_type == 'SLSQP':
    optimizer = SLSQP(maxiter=5000)
    

In [7]:
from qiskit.algorithms import VQE
from IPython.display import display, clear_output

# Print and save the data in lists
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)

counts = []
values = []
params = []
deviation = []

# Set initial parameters of the ansatz
# We choose a fixed small displacement 
# So all participants start from similar starting point
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters

algorithm = VQE(ansatz,
                optimizer=optimizer,
                quantum_instance=backend,
                callback=callback,
                initial_point=initial_point)

result = algorithm.compute_minimum_eigenvalue(qubit_op)

print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 529,
    'eigenstate': array([-1.42292337e-03+3.61807313e-05j, -1.93202168e-07-1.61578454e-06j,
        2.41260073e-02+1.37355712e-04j, -9.91413874e-01-5.89548510e-03j,
       -5.41133554e-02+4.36415228e-04j, -6.27799952e-06-6.15481838e-05j,
        6.34550548e-04-7.40154454e-06j, -2.60758018e-02+2.97549067e-04j,
        2.98098256e-03-6.80889365e-05j,  7.03112981e-05-1.64006552e-06j,
       -1.55225856e-03-2.36671971e-05j, -5.44839762e-04+9.82353580e-04j,
        1.13360799e-01-6.21249464e-04j,  2.67382041e-03-1.59485695e-05j,
       -4.08335222e-05+8.62055393e-08j, -1.38802375e-05+2.60835613e-05j]),
    'eigenvalue': -1.0863420218744921,
    'optimal_parameters': {   ParameterVectorElement(θ[3]): 0.009997878564026579,
                              ParameterVectorElement(θ[4]): 3.3936472368330555,
                              ParameterVectorElement(θ[5]): 0.015077877454169346,
                              ParameterVect

In [8]:
# Check your answer using following code
from qc_grader import grade_ex5
freeze_core = True # change to True if you freezed core electrons
grade_ex5(ansatz,qubit_op,result,freeze_core)

Grading your answer for ex5. Please wait...

Congratulations 🎉! Your answer is correct.
Your cost is 3.
Feel free to submit your answer.



In [11]:
energy = result.optimal_value
energy

-1.0863420218744921

In [12]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
print(result_exact)

# The targeted electronic energy for H2 is -1.85336 Ha
# Check with your VQE result.

Exact electronic energy -1.0887060157347404
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.907396311316
  - computed part:      -1.088706015735
  - FreezeCoreTransformer extracted energy part: -7.818690295581
~ Nuclear repulsion energy (Hartree): 1.025934879643
> Total ground state energy (Hartree): -7.881461431673
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  2.92416221]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  4.76300889]
    - computed part:      [0.0  0.0  4.76695575]
    - FreezeCoreTransformer extracted energy part: [0.0  0.0  -0.00394686]
  > Dipole moment (a.u.): [0.0  0.0  -1.83884668]  Total: 1.83884668
                 (debye): [0.0  0.0  -4.67388163]  Total: 4.67388163
 


In [14]:
# Chemical Accuracy in mHa
error_mHa= (energy-exact_energy)*1000
print(error_mHa)

2.3639938602482147
